# Tensor Network documentation tutorial

In [8]:
# import necessary packages
import numpy as np
import tensornetwork as tn

#### Nodes

In [9]:
node = tn.Node(np.eye(2))
print(node.tensor)  # This is how you access the underlying tensor.

[[1. 0.]
 [0. 1.]]


#### Edges

In [10]:
a = tn.Node(np.eye(2))
b = tn.Node(np.eye(2))
c = tn.Node(np.eye(2))
# Dangling edges are automatically created at node creation.
# We can access them this way.
dangling_edge = a.get_edge(1)
# This is the same as above
dangling_edge = a[1]
# Create a standard edge by connecting any two separate nodes together.
# We create a new edge by "connecting" two dangling edges.
# standard_edge = a[0] ^ b[0] # same as tn.connect(a[0], b[0])
standard_edge = tn.connect(a[0], b[0])
# Create a trace edge by connecting a node to itself.
trace_edge = c[0] ^ c[1]

In [11]:
def one_edge_at_a_time(a, b):
  node1 = tn.Node(a)
  node2 = tn.Node(b)
  edge1 = node1[0] ^ node2[0]
  edge2 = node1[1] ^ node2[1]
  tn.contract(edge1)
  result = tn.contract(edge2)
  return result.tensor

def use_contract_between(a, b):
  node1 = tn.Node(a)
  node2 = tn.Node(b)
  node1[0] ^ node2[0]
  node1[1] ^ node2[1]
  # This is the same as
  # tn.contract_between(node1, node2)
  result = node1 @ node2
  return result.tensor

a = np.ones((1000, 1000))
b = np.ones((1000, 1000))

res1 = one_edge_at_a_time(a, b)
print(res1)
res2 = use_contract_between(a, b)
print(res2)


1000000.0
1000000.0


### Matrix Product States (MPS)

In [32]:
def block(*dimensions):
  '''Construct a new matrix for the MPS with random numbers from 0 to 1'''
  size = tuple([x for x in dimensions])
  return np.random.random_sample(size)


def create_MPS(rank, dim, bond_dim):
  '''Build the MPS tensor'''
  mps = (
    [tn.Node( block(dim, bond_dim) )] +
    [tn.Node( block(bond_dim, dim, bond_dim)) for _ in range(rank-2)] +
    [tn.Node( block(bond_dim, dim) )]
  )

  #connect edges to build mps
  connected_edges=[]
  conn=mps[0][1]^mps[1][0]
  connected_edges.append(conn)
  for k in range(1,rank-1):
      conn=mps[k][2]^mps[k+1][0]
      connected_edges.append(conn)

  return mps, connected_edges

In [36]:
########################################################################
#----- Retrieving a Component from an MPS by Contracting its edges-----#
########################################################################
dim = 2
bond_dim = 2
rank = 10
components=tuple(np.random.randint(0, dim, rank)) #select randomly the components that we will retrieve
print(f'components are: {components}')

mps_nodes, mps_edges = create_MPS(rank, dim, bond_dim)
for k in range(len(mps_edges)):
    A = tn.contract(mps_edges[k])

#the last node now has all the edges corresponding to the tensor components.

print(f'coefficient of the tensor at the selected components: {A.tensor[components]}')


components are: (0, 0, 0, 1, 1, 0, 1, 0, 1, 1)
coefficient of the tensor at the selected components: 0.9827227855288356


In [56]:
A = tn.Node(np.random.random_sample((2,2,2)), name='a')
A.shape
print(A[0])


__unnamed_edge__
